### json 불러와서 저장하기

In [1]:
import numpy as np
import pandas as pd
import json
import jpype
import glob
from random import shuffle
from bs4 import BeautifulSoup as bs

#data_path = "data"
data_path = "data(test)"

file_list = glob.glob("%s/*.json" % data_path)
shuffle(file_list)

json_train=[]
for json_file_name in file_list:
    json_file = json.loads(open(json_file_name).read())
    json_train += json_file["articles"]

### json에서 text 형태소 추출

In [10]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    text = bs(article["text"], "html.parser").text
    
    if cnt % 10000 == 0:
        print(cnt)
        
    text_pos = ["%s_%s" % (first, second) for first, second in mecab.pos(text)]

    labeled_train.append({
            "istroll": article["is_troll"],
            "text_pos": text_pos,
            "text_pos_sentences" : " ".join(text_pos),
            "pk": article["pk"]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train = labeled_train.set_index('pk')

labeled_train

0


,istroll,text_pos,text_pos_sentences
pk,,,
1,False,"[일간_NNG, 워스트_NNP, 마을_NNG, 회관_NNG, 입니다_VCP+EF, ...",일간_NNG 워스트_NNP 마을_NNG 회관_NNG 입니다_VCP+EF ._SF 공...
2,False,"[<_SY, 일간_NNG, 워스트_NNP, >_SY, 입니다_NNB+VCP+EF, ...",<_SY 일간_NNG 워스트_NNP >_SY 입니다_NNB+VCP+EF 추천_NNG...
3,False,"[벌써_MAG, 열린_VV+ETM, 거_NNB, 에요_VCP+EF, !_SF, ?_SF]",벌써_MAG 열린_VV+ETM 거_NNB 에요_VCP+EF !_SF ?_SF
4,False,"[아햏햏_NNP, 고구마_NNG, 장사_NNG, 가_JKS, 힘들_VA, 어요_EF]",아햏햏_NNP 고구마_NNG 장사_NNG 가_JKS 힘들_VA 어요_EF
5,False,"[ㅎ_XSA, ㄷ_NNG, ㄷ_NNG, 사스_NNP, 가_JKS]",ㅎ_XSA ㄷ_NNG ㄷ_NNG 사스_NNP 가_JKS
6,False,"[민영화_NNG, 는_JX, 가_VV, 라_EF, !_SF, 우리_NP, 는_JX,...",민영화_NNG 는_JX 가_VV 라_EF !_SF 우리_NP 는_JX 안녕_NNG ...
7,False,"[제_NP, 가_JKS, 친_VV+ETM, 드립_NNG, 이_JKS, 세_MM, 시...",제_NP 가_JKS 친_VV+ETM 드립_NNG 이_JKS 세_MM 시간_NNG 뒤...
8,False,"[신장_NNG, 팔_VV, 아_EC, 간다_VX+EC]",신장_NNG 팔_VV 아_EC 간다_VX+EC
9,False,"[페이스북_NNP, 디자인_NNG, 카피_NNG, 하_XSV, 고_EC, 좋_VA,...",페이스북_NNP 디자인_NNG 카피_NNG 하_XSV 고_EC 좋_VA 아요_EF ...


### bag of words

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)
    
    train_data_features = vectorizer.fit_transform(labeled_train[col_name]).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    #col = ["bow%s%s" % (col_name, str(i)) for  in range(0, max_features)]
    df_bow = pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)
    
    labeled_train = pd.concat([labeled_train, df_bow],axis=1)
    
    return labeled_train

## LDA

In [8]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    
    for i in range(0,num):
        if i % 10000 == 0:
            print(i)
        
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = train.index
    
    train = pd.concat([train, df], axis=1)
        
    return train

In [13]:
max_features = 1000
#%time labeled_train = make_lda(labeled_train, 5000, 1000, "text_pos")
%time labeled_train = make_bag_of_words(labeled_train, 1000, "text_pos_sentences")

CPU times: user 14.6 ms, sys: 1.51 ms, total: 16.2 ms
Wall time: 15.4 ms


### predictor, model 생성

In [16]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['text_pos', 'text_pos_sentences', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

### cross validation

In [17]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/Library/Frameworks/Python.f

IndexError: index 1 is out of bounds for axis 1 with size 1

##Result
  * text의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.637526911627
  * text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.661265083065
  * 데이터를 셔플하고 text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.688875771784
  
  * text의 pos를 lda로 변환 (keep_n : 5,000개, num_topics : 1,000개) - 0.643143549702
  * bs의 파라미터를 "lxml"이 아닌 "html.parser"로 주고 pos를 bag of words로 변환 (feature : 1,000개) - 0.657726939833

In [18]:
import pickle

pickle.dump(labeled_train[pre], open("text_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)